In [234]:
import pandas as pd 
import numpy as np 
from scipy import stats
from matplotlib import pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta

In [235]:
locatel = pd.read_csv('C:/Users/Tania/Downloads/base-integrale-140322.csv')

In [236]:
locatel.shape

(1048575, 28)

In [237]:
locatel.head()

,FOLIO,FECHA_ALTA,A�O_ALTA,MES_ALTA,D�A_ALTA,HORA_ALTA,SEXO,EDAD,ESTADO_CIVIL,OCUPACION,...,CP_HECHOS,ORIGEN,SERVICIO,TEMATICA_1,TEMATICA_2,TEMATICA_3,TEMATICA_4,TEMATICA_5,TEMATICA_6,TEMATICA_7
0,3011.0,2016-11-01,2016,11,1,0,MASCULINO,34,CASADO (A),EMPLEADO (A),...,11260.0,ASESOR�A,JUR�DICO,INFORMACION GENERAL,FUNDAMENTOS LEGALES,NaN,NaN,NaN,NaN,NaN
1,3012.0,2016-11-01,2016,11,1,0,FEMENINO,54,SOLTERO (A),EMPLEADO (A),...,NaN,ASESOR�A,JUR�DICO,INFORMACION GENERAL,DEPENDENCIAS ADMINISTRATIVAS,REFERENCIA INSTITUCIONAL,NaN,NaN,NaN,NaN
2,3013.0,2016-11-01,2016,11,1,0,FEMENINO,42,SOLTERO (A),EMPLEADO (A),...,7330.0,ASESOR�A,JUR�DICO,PENAL,DELITOS CONTRA LA LIBERTAD Y LA SEGURIDAD SEXU...,ABUSO SEXUAL,MENORES DE EDAD,NaN,NaN,NaN
3,3014.0,2016-11-01,2016,11,1,1,FEMENINO,40,SOLTERO (A),DESEMPLEADO (A),...,2100.0,ASESOR�A,JUR�DICO,PENAL,DELITOS CONTRA LA INTEGRIDAD FAMILIAR,VIOLENCIA FAMILIAR,NaN,NaN,NaN,NaN
4,3015.0,2016-11-01,2016,11,1,1,FEMENINO,55,SOLTERO (A),EMPLEADO (A),...,NaN,ASESOR�A,JUR�DICO,INFORMACION GENERAL,GARANTIAS DEL GOBERNADO,NaN,NaN,NaN,NaN,NaN


In [238]:
locatel.FECHA_ALTA.max()

'2022-02-13'

In [239]:
locatel["FECHA_ALTA"] = pd.to_datetime(locatel["FECHA_ALTA"])

In [240]:
localtel_diario = locatel.groupby(["FECHA_ALTA", "TEMATICA_1"])["FOLIO"].count()

In [241]:
localtel_diario = localtel_diario.xs("VIOLENCIA", level = 1)

In [242]:
localtel_diario = localtel_diario.to_frame().reset_index()

In [243]:
import datetime

In [244]:
format = "%Y-%m-%d"
fecha_covid = datetime.datetime.strptime("2020-03-13", format)
fecha_covid

datetime.datetime(2020, 3, 13, 0, 0)

In [245]:
localtel_diario['covid'] = localtel_diario["FECHA_ALTA"].apply(lambda fecha: 1 if fecha > fecha_covid else 0)

In [250]:
#Base de datos de restricciones covid 
restricciones = pd.read_csv('C:/Users/Tania/Downloads/covid-stringency-index.csv')

In [251]:
restricciones.head()

,Entity,Code,Day,stringency_index
0,Afghanistan,AFG,2020-01-21,0.0
1,Afghanistan,AFG,2020-01-22,0.0
2,Afghanistan,AFG,2020-01-23,0.0
3,Afghanistan,AFG,2020-01-24,0.0
4,Afghanistan,AFG,2020-01-25,0.0


In [252]:
restricciones=restricciones[restricciones.Code=="MEX"]

In [253]:
restricciones.head()

,Entity,Code,Day,stringency_index
85377,Mexico,MEX,2020-01-21,0.0
85378,Mexico,MEX,2020-01-22,0.0
85379,Mexico,MEX,2020-01-23,0.0
85380,Mexico,MEX,2020-01-24,0.0
85381,Mexico,MEX,2020-01-25,0.0


In [254]:
restricciones=restricciones[["Day","stringency_index"]].rename({"Day":"FECHA_ALTA"}, axis = 1)

In [255]:
restricciones["FECHA_ALTA"] = pd.to_datetime(restricciones["FECHA_ALTA"])

In [256]:
restricciones.FECHA_ALTA.max()

Timestamp('2022-03-20 00:00:00')

In [257]:
restricciones[restricciones.FECHA_ALTA>"2022-01-01"]

,FECHA_ALTA,stringency_index
86089,2022-01-02,31.48
86090,2022-01-03,31.48
86091,2022-01-04,31.48
86092,2022-01-05,31.48
86093,2022-01-06,31.48
...,...,...
86162,2022-03-16,35.19
86163,2022-03-17,35.19
86164,2022-03-18,35.19
86165,2022-03-19,35.19


In [258]:
localtel_diario = pd.merge(localtel_diario, restricciones, how = "left", left_on = "FECHA_ALTA", right_on = "FECHA_ALTA")

In [259]:
localtel_diario = localtel_diario.rename({"FOLIO":"llamadas"}, axis =1)

In [260]:
localtel_diario.isna().sum()

FECHA_ALTA             0
llamadas               0
covid                  0
stringency_index    1165
dtype: int64

In [261]:
localtel_diario.stringency_index = localtel_diario.stringency_index.fillna(0)

In [262]:
restricciones_tot=localtel_diario[["FECHA_ALTA","stringency_index"]]

In [263]:
restricciones_tot.to_csv("restricciones_tot.csv")

In [264]:
#Identificamos en que día de la semana y mus del año se hicieron las llamadas 
localtel_diario["dia"] = localtel_diario["FECHA_ALTA"].apply(lambda fecha: pd.Timestamp(f"{fecha}").day_name())
localtel_diario["mes"] = localtel_diario["FECHA_ALTA"].apply(lambda fecha: pd.Timestamp(f"{fecha}").month_name())

In [265]:
localtel_diario.groupby("covid")["llamadas"].mean()

covid
0    13.296388
1    32.015670
Name: llamadas, dtype: float64

In [266]:
localtel_diario.groupby("dia")["llamadas"].mean()

dia
Friday       21.003636
Monday       22.430657
Saturday     16.545788
Sunday       16.076364
Thursday     21.083636
Tuesday      22.058394
Wednesday    21.781022
Name: llamadas, dtype: float64

In [267]:
localtel_diario.groupby("mes")["llamadas"].mean()

mes
April        28.813333
August       20.600000
December     14.972826
February     18.256757
January      15.102151
July         19.741935
June         20.673333
March        26.154839
May          21.619355
November     17.491525
October      20.929032
September    19.886667
Name: llamadas, dtype: float64

In [275]:
localtel_diario

,FECHA_ALTA,llamadas,covid,stringency_index,dia,mes
0,2016-11-04,5,0,0.00,Friday,November
1,2016-11-05,1,0,0.00,Saturday,November
2,2016-11-06,6,0,0.00,Sunday,November
3,2016-11-07,13,0,0.00,Monday,November
4,2016-11-08,11,0,0.00,Tuesday,November
...,...,...,...,...,...,...
1915,2022-02-09,28,1,35.19,Wednesday,February
1916,2022-02-10,12,1,35.19,Thursday,February
1917,2022-02-11,20,1,35.19,Friday,February
1918,2022-02-12,11,1,35.19,Saturday,February


In [276]:
localtel_diario.to_csv("dias.csv")

In [29]:
localtel_diario["lag1"] = localtel_diario.llamadas.shift(1)

In [30]:
localtel_diario = pd.get_dummies(data=localtel_diario, drop_first=False)

In [31]:
localtel_diario = localtel_diario.drop(0)

In [32]:
localtel_diario.to_csv("locatel_diario.csv")

**Intento1, usaremos como variable el indice de restriccion por pandemia**

In [57]:
pred_vars = ['stringency_index', 'lag1', 'dia_Monday', 'dia_Saturday',
       'dia_Friday', 'dia_Thursday', 'dia_Tuesday', 'dia_Wednesday',
       'mes_August', 'mes_April', 'mes_February', 'mes_January', 'mes_July',
       'mes_June', 'mes_March', 'mes_May', 'mes_November', 'mes_October',
       'mes_September']

In [58]:
X = localtel_diario[pred_vars]

In [59]:
X

,stringency_index,lag1,dia_Monday,dia_Saturday,dia_Friday,dia_Thursday,dia_Tuesday,dia_Wednesday,mes_August,mes_April,mes_February,mes_January,mes_July,mes_June,mes_March,mes_May,mes_November,mes_October,mes_September
1,0.00,5.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0.00,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0.00,6.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0.00,13.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
5,0.00,11.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,35.19,16.0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0
1916,35.19,28.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1917,35.19,12.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1918,35.19,20.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [60]:
y = localtel_diario.llamadas

In [61]:
y.describe()

count    1919.000000
mean       20.148515
std        13.672214
min         1.000000
25%        11.000000
50%        16.000000
75%        26.500000
max       102.000000
Name: llamadas, dtype: float64

In [62]:
X_train=X[0:1739]
X_test=X[1739:]

In [63]:
y_train=y[0:1739]
y_test=y[1739:]

In [64]:
X = sm.add_constant(X, prepend=False)

# Fit and summarize OLS model
mod = sm.OLS(y, X)

res = mod.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:               llamadas   R-squared:                       0.717
Model:                            OLS   Adj. R-squared:                  0.714
Method:                 Least Squares   F-statistic:                     252.9
Date:                Fri, 25 Mar 2022   Prob (F-statistic):               0.00
Time:                        10:47:31   Log-Likelihood:                -6531.1
No. Observations:                1919   AIC:                         1.310e+04
Df Residuals:                    1899   BIC:                         1.321e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
stringency_index     0.1366      0.008  

In [65]:
localtel_diario["predict_restriccion"]= res.predict()

In [66]:
np.sqrt(np.sum((localtel_diario.llamadas - localtel_diario.predict_restriccion)**2))

318.6858149415731

In [67]:
linreg = LinearRegression()

In [68]:
linreg.fit(X_train, y_train)

LinearRegression()

In [69]:
linreg.score(X_train, y_train)

0.7372639223485606

In [70]:
predict_train=linreg.predict(X_train)

In [71]:
mean_squared_error(y_train, predict_train)

51.1412504415065

In [72]:
linreg.score(X_test,y_test)

0.3205356673201982

In [73]:
predict_test=linreg.predict(X_test)

In [74]:
mean_squared_error(y_test, predict_test)

71.3694235839471

In [75]:
predicciones=pd.concat([pd.DataFrame(predict_train), pd.DataFrame(predict_test)], axis=0)
predicciones.reset_index(drop=True,inplace=True)


In [76]:
llamadas=localtel_diario[["FECHA_ALTA","llamadas"]]

In [77]:
llamadas.reset_index(drop=True,inplace=True)

In [78]:
predict=pd.concat([llamadas, predicciones], axis=1,)

In [288]:
predict.to_csv("predict1.csv")

In [225]:
#Aunque tenemos una buena aproximación de los datos usando el indice covid, este es desconocido para nosotros en los
#siguientes dáias que queramos predecir, por lo que vamos a intentar hacer un modelo un poquito distinto

# Otro  Modelo

In [79]:
#En lugar de usar el indice de restricción usaremos un indice covid como variable en la que es cero si no habia 
#precencia de covid en ese día y 1 si sí.
pred_vars1 = ['covid', 'lag1', 'dia_Monday', 'dia_Saturday',
       'dia_Friday', 'dia_Thursday', 'dia_Tuesday', 'dia_Wednesday',
       'mes_August', 'mes_April', 'mes_February', 'mes_January', 'mes_July',
       'mes_June', 'mes_March', 'mes_May', 'mes_November', 'mes_October',
       'mes_September']

In [80]:
X = localtel_diario[pred_vars1]

In [81]:
y = localtel_diario.llamadas

In [172]:
X.shape

(1919, 20)

In [171]:
X = sm.add_constant(X, prepend=False)

# Fit and summarize OLS model
mod = sm.OLS(y, X).fit()

print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:               llamadas   R-squared:                       0.710
Model:                            OLS   Adj. R-squared:                  0.707
Method:                 Least Squares   F-statistic:                     245.1
Date:                Fri, 25 Mar 2022   Prob (F-statistic):               0.00
Time:                        12:16:07   Log-Likelihood:                -6552.6
No. Observations:                1919   AIC:                         1.315e+04
Df Residuals:                    1899   BIC:                         1.326e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
covid             7.2803      0.490     14.871

In [87]:
np.sqrt(np.sum((localtel_diario.llamadas - localtel_diario.predict_restriccion)**2))

318.6858149415731

In [173]:
X_train=X[0:1739]
X_test=X[1739:]

In [174]:
y_train=y[0:1739]
y_test=y[1739:]

In [175]:
linreg = LinearRegression()

In [177]:
linreg.fit(X_train, y_train)

LinearRegression()

In [178]:
linreg.score(X_train, y_train)

0.7368374075231485

In [179]:
predict_train=linreg.predict(X_train)

In [180]:
mean_squared_error(y_train, predict_train)

51.22427102131568

In [181]:
linreg.score(X_test,y_test)

0.13467773235454172

In [182]:
predict_test=linreg.predict(X_test)

In [183]:
mean_squared_error(y_test, predict_test)

90.89152805510639

In [186]:
linreg.coef_

array([ 8.99432318,  0.58310035,  6.55189637, -1.83879349,  2.4029582 ,
        2.11279019,  2.66743238,  2.48762529,  1.33448595,  4.53027726,
        2.22843841,  0.82889018,  0.6941862 ,  1.04076527,  4.17518682,
        1.46172146,  1.03681582,  1.64862309,  1.63515089,  0.        ])

In [187]:
predicciones=pd.concat([pd.DataFrame(predict_train), pd.DataFrame(predict_test)], axis=0)
predicciones.reset_index(drop=True,inplace=True)


In [188]:
llamadas=localtel_diario[["FECHA_ALTA","llamadas"]]

In [189]:
llamadas.reset_index(drop=True,inplace=True)

In [190]:
predict=pd.concat([llamadas, predicciones], axis=1,)

In [264]:
predict.to_csv("predict.csv")

In [191]:
X

,covid,lag1,dia_Monday,dia_Saturday,dia_Friday,dia_Thursday,dia_Tuesday,dia_Wednesday,mes_August,mes_April,mes_February,mes_January,mes_July,mes_June,mes_March,mes_May,mes_November,mes_October,mes_September,const
1,0,5.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1.0
2,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1.0
3,0,6.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1.0
4,0,13.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1.0
5,0,11.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,1,16.0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1.0
1916,1,28.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0
1917,1,12.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0
1918,1,20.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0


**Predicciones modelo con indice covid**

In [101]:
#Obtendrémos las predicciones de varios días en adelante 
predict

,FECHA_ALTA,llamadas,0
0,2016-11-05,1,3.941824
1,2016-11-06,6,3.448216
2,2016-11-07,13,12.915614
3,2016-11-08,11,13.112852
4,2016-11-09,11,11.766845
...,...,...,...
1914,2022-02-09,28,24.868292
1915,2022-02-10,12,31.490661
1916,2022-02-11,20,22.451224
1917,2022-02-12,11,22.874275


In [115]:
inicio=datetime(2022,2,14)
fin=datetime(2022,6,14)
lista_fechas = [inicio + timedelta(days=d) for d in range((fin - inicio).days + 1)] 

In [116]:
nuevos = pd.DataFrame()

In [117]:
nuevos["FECHA_ALTA"]=lista_fechas

In [118]:
nuevos['covid'] = nuevos["FECHA_ALTA"].apply(lambda fecha: 1 if fecha > fecha_covid else 0)

In [121]:
nuevos["dia"] = nuevos["FECHA_ALTA"].apply(lambda fecha: pd.Timestamp(f"{fecha}").day_name())
nuevos["mes"] = nuevos["FECHA_ALTA"].apply(lambda fecha: pd.Timestamp(f"{fecha}").month_name())

In [122]:
nuevos = pd.get_dummies(data=nuevos, drop_first=False)

In [135]:
nuevos["mes_August"]=0
nuevos["mes_September"]=0
nuevos["mes_October"]=0
nuevos["mes_November"]=0
nuevos["mes_July"]=0
nuevos["mes_January"]=0


In [138]:
nuevos=nuevos.reindex(columns=["FECHA_ALTA","covid","dia_Monday","dia_Saturday","dia_Friday","dia_Thursday","dia_Tuesday","dia_Wednesday","mes_August","mes_April","mes_February","mes_January","mes_July","mes_June","mes_March","mes_May","mes_November","mes_October","mes_September","dia_Sunday",])

In [139]:
pred_vars = ['covid', 'dia_Monday', 'dia_Saturday',
       'dia_Friday', 'dia_Thursday', 'dia_Tuesday', 'dia_Wednesday',
       'mes_August', 'mes_April', 'mes_February', 'mes_January', 'mes_July',
       'mes_June', 'mes_March', 'mes_May', 'mes_November', 'mes_October',
       'mes_September']

In [142]:
X_n=nuevos[pred_vars]
X_n["const"]=1.0

<ipython-input-142-57ff530f8ddb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_n["const"]=1.0


In [166]:
X_n.shape

(121, 20)

In [167]:
X.shape

(1919, 20)

In [144]:
X_n.insert(1, 'lag1', np.nan)

In [148]:
X_n.loc[0, "lag1"] = 8

In [164]:
#X_n.loc[0].values.reshape(1,-1)

array([[1., 8., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 1.]])

In [194]:
X_n[X_n.index==0]

,covid,lag1,dia_Monday,dia_Saturday,dia_Friday,dia_Thursday,dia_Tuesday,dia_Wednesday,mes_August,mes_April,mes_February,mes_January,mes_July,mes_June,mes_March,mes_May,mes_November,mes_October,mes_September,const
0,1,8.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0


In [196]:
X_n.index

RangeIndex(start=0, stop=121, step=1)

In [197]:
nuevas_pred=pd.DataFrame()

In [200]:
nuevas_pred["Fecha"]=lista_fechas
nuevas_pred["predicciones"]=np.nan

In [201]:
for i in X_n.index:
    pre=linreg.predict(X_n[X_n.index==i])
    nuevas_pred.loc[i,"predicciones"]=pre
    if i!=len(X_n.index)-1:
        X_n.loc[i+1, "lag1"] = pre
    
    
    

In [203]:
X_n

,covid,lag1,dia_Monday,dia_Saturday,dia_Friday,dia_Thursday,dia_Tuesday,dia_Wednesday,mes_August,mes_April,mes_February,mes_January,mes_July,mes_June,mes_March,mes_May,mes_November,mes_October,mes_September,const
0,1,8.000000,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0
1,1,24.267760,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1.0
2,1,29.869033,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1.0
3,1,32.955330,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0
4,1,34.380116,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,1,34.338457,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1.0
117,1,34.289113,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1.0
118,1,30.018588,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1.0
119,1,29.367237,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1.0


In [202]:
nuevas_pred

,Fecha,predicciones
0,2022-02-14,24.267760
1,2022-02-15,29.869033
2,2022-02-16,32.955330
3,2022-02-17,34.380116
4,2022-02-18,35.501077
...,...,...
116,2022-06-10,34.289113
117,2022-06-11,30.018588
118,2022-06-12,29.367237
119,2022-06-13,35.539331


In [204]:
nuevas_pred.insert(1, 'llamadas', np.nan)

In [205]:
nuevas_pred

,Fecha,llamadas,predicciones
0,2022-02-14,NaN,24.267760
1,2022-02-15,NaN,29.869033
2,2022-02-16,NaN,32.955330
3,2022-02-17,NaN,34.380116
4,2022-02-18,NaN,35.501077
...,...,...,...
116,2022-06-10,NaN,34.289113
117,2022-06-11,NaN,30.018588
118,2022-06-12,NaN,29.367237
119,2022-06-13,NaN,35.539331


In [211]:
predict.columns=["Fecha","llamadas","predicciones"]

In [212]:
predict

,Fecha,llamadas,predicciones
0,2016-11-05,1,3.941824
1,2016-11-06,6,3.448216
2,2016-11-07,13,12.915614
3,2016-11-08,11,13.112852
4,2016-11-09,11,11.766845
...,...,...,...
1914,2022-02-09,28,24.868292
1915,2022-02-10,12,31.490661
1916,2022-02-11,20,22.451224
1917,2022-02-12,11,22.874275


In [215]:
resultados=pd.concat([predict, nuevas_pred], axis=0)

In [217]:
resultados.reset_index(inplace=True,drop=True)

In [218]:
resultados

,Fecha,llamadas,predicciones
0,2016-11-05,1.0,3.941824
1,2016-11-06,6.0,3.448216
2,2016-11-07,13.0,12.915614
3,2016-11-08,11.0,13.112852
4,2016-11-09,11.0,11.766845
...,...,...,...
2035,2022-06-10,NaN,34.289113
2036,2022-06-11,NaN,30.018588
2037,2022-06-12,NaN,29.367237
2038,2022-06-13,NaN,35.539331


In [219]:
resultados.to_csv("resultados.csv")

In [226]:
resultados1=resultados[(resultados.Fecha>="2022-02-14")&(resultados.Fecha<="2022-02-14")]

In [227]:
resultados1.predicciones.mean()

24.267760377553703